In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Sentiment Analysis/emoevent_es.csv'
df = pd.read_csv(file_path, delimiter='\t')
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized = tokenizer(df['tweet'].tolist(), max_length=128, padding='max_length', truncation=True, return_tensors='tf')

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = tokenized['input_ids'].numpy()
y = df['emotion'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=16, class_weight=class_weights_dict, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/5
379/379 [==============================] - 230s 471ms/step - loss: 1.9717 - accuracy: 0.1267 - val_loss: 2.0288 - val_accuracy: 0.0163
Epoch 2/5
379/379 [==============================] - 179s 472ms/step - loss: 1.9860 - accuracy: 0.1049 - val_loss: 1.9444 - val_accuracy: 0.1085
Epoch 3/5
379/379 [==============================] - 178s 471ms/step - loss: 1.9858 - accuracy: 0.1092 - val_loss: 1.9636 - val_accuracy: 0.0074
Epoch 4/5
379/379 [==============================] - 178s 471ms/step - loss: 1.9860 - accuracy: 0.1001 - val_loss: 1.9035 - val_accuracy: 0.0446
Epoch 5/5
379/379 [==============================] - 174s 459ms/step - loss: 1.9861 - accuracy: 0.1135 - val_loss: 2.0667 - val_accuracy: 0.0074


In [ ]:
predictions = model.predict(X_test).logits
predicted_labels = [np.argmax(p) for p in predictions]
print(classification_report(y_test, predicted_labels))

53/53 [==============================] - 19s 275ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       180
           1       0.00      0.00      0.00        30
           2       0.01      1.00      0.02        13
           3       0.00      0.00      0.00       332
           4       0.00      0.00      0.00       835
           5       0.00      0.00      0.00       229
           6       0.00      0.00      0.00        63

    accuracy                           0.01      1682
   macro avg       0.00      0.14      0.00      1682
weighted avg       0.00      0.01      0.00      1682



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
